In [1]:
__author__ = 'Monika Soraisam'
__email__ = 'monika.soraisam@noirlab.edu'

# This notebook is meant for generating trimmed files for unit testing for Dan (https://noirlab.atlassian.net/jira/software/projects/GOATS/boards/57?selectedIssue=GOATS-372)

In [2]:
import os
import sys
from pathlib import Path
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
import types

In [3]:
# Now import the DRAGONS libraries 
import astrodata
import gemini_instruments
from recipe_system import cal_service
from gempy.adlibrary import dataselect
from gempy.utils import logutils

from recipe_system.reduction.coreReduce import Reduce
from gempy.scripts import showpars
from gempy.utils.showrecipes import showrecipes
from gempy.utils.showrecipes import showprims

In [4]:
## prep the reduction folder
def prep_reduction_folder(data_root, obsid):
    reduction_path = Path(f"{data_root}/{obsid}/reduction")

    if not reduction_path.exists():
        os.mkdir(reduction_path.as_posix())
        print (f"directory for dragons reduction created")
    
    ## change the cwd to the reduction folder
    os.chdir(reduction_path.as_posix())
    print(f"Current working directory is: {os.getcwd()}")

    ## write the configuration file 
    mydb = "dragons_for_goats.db" 
    mydb_path = reduction_path.as_posix() + '/' + mydb
    print (mydb_path)
    
    dragons_rc = reduction_path.as_posix() + '/dragonsrc'
    print (dragons_rc)
    
    with open(dragons_rc, "w") as f:
        f.write("[calibs]\ndatabases = {0} get store".format(mydb_path))

    return dragons_rc, mydb_path


In [5]:
data_path = "/Users/monika.soraisam/Desktop/tomdev/real_goats"
obsid = 'GS-2021A-DD-102-9'  # GMOS longslit example 
dragons_rc, mydb_path = prep_reduction_folder(data_path, obsid)


Current working directory is: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/reduction
/Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/reduction/dragons_for_goats.db
/Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/reduction/dragonsrc


In [6]:
def generate_filelists(location, obsid):
    """
    Parameters
    ----------
    location: str
        Root folder where the Gemini data for a given target is located
    obsid: str
        Gemini observation ID 
    """

    
    all_files = [str(pp) for pp in list(Path(location+"/"+obsid).glob('*.fits'))]
    all_files.sort()
    print (f'The total number of files for observation ID {obsid} is {len(all_files)}')

    obs_types = ['OBJECT','BIAS','DARK','FLAT','ARC','PINHOLE','RONCHI','CAL','FRINGE','MOS MASK', 'BPM'] #fetched from Obs Type search field on GOA, which is relevant for DRAGONS

    all_meta = {}

    # Note that even if a few keywords are extracted below, thiis is just meant for demonstration purposes. 
    # If onyl these keywords are extracted in production, it will not align with  
    # implementation of item (2), allowing users to group/select preferred header keywords
    for K in obs_types:
        all_meta[K] = {'file':[],
                        'obs_class':[],
                        'group_id':[],
                        'exp':[],
                        'object':[],
                        'wave':[],
                        'waveband':[],
                        'date':[],
                        'roi':[],
                        }

    object_files = []
    for i,F in enumerate(all_files):
        ad = astrodata.open(F)

        if "BPM" in ad.tags or "UNPREPARED" in ad.tags: ## want only raw, i.e., "unprepared" files, but BPM is an exception, which is processed/prepared 
            K = ad.observation_type() ## astrodata header also has the observation type, which should match what's in the archive drop-down menu
        
        elif "PREPARED" in ad.tags or "PROCESSED" in ad.tags: ## skip all "prepared"/"processed" files
            continue
            
        all_meta[K]['file'].append(F)
        all_meta[K]['obs_class'].append(ad.observation_class())
        
        # group_id seems to be not implemented for GNIRS spectroscopy yet
        if "GNIRS" in ad.instrument():
            all_meta[K]['group_id'].append(None)
        else:
            all_meta[K]['group_id'].append(ad.group_id())
        all_meta[K]['exp'].append(ad.exposure_time())
        all_meta[K]['object'].append(ad.object())
        all_meta[K]['wave'].append(ad.central_wavelength())
        all_meta[K]['waveband'].append(ad.wavelength_band())
        all_meta[K]['date'].append(ad.ut_date())
        all_meta[K]['roi'].append(ad.detector_roi_setting()) 
        #print (F.split('/')[-1], ad.object(), ad.tags)
    
    return all_meta


In [7]:
all_meta = generate_filelists(data_path, obsid)

for K,V in all_meta.items():
    if len(V['file'])==0:
        continue
    print (f"There are {len(V['file'])} files for observation type {K}") 

The total number of files for observation ID GS-2021A-DD-102-9 is 59
There are 4 files for observation type OBJECT
There are 50 files for observation type BIAS
There are 2 files for observation type FLAT
There are 2 files for observation type ARC
There are 1 files for observation type BPM


In [8]:
DF_bias = pd.DataFrame(all_meta['BIAS'])
DF_flat = pd.DataFrame(all_meta['FLAT'])
DF_bpm = pd.DataFrame(all_meta['BPM'])
DF_object = pd.DataFrame(all_meta['OBJECT'])
DF_arc = pd.DataFrame(all_meta['ARC'])

In [9]:
print (f"There are {len(np.unique(DF_object['object'].values))} science targets in this observation set, namely {np.unique(DF_object['object'].values)}")

There are 2 science targets in this observation set, namely ['AT2020caa' 'Twilight']


## We will use astrodata to "write" the trimmed arrary files for each observation type but retaining the same header info. 

In [12]:
## for bias

ad = astrodata.open(DF_bias['file'].values[0])
ad.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/S20210211S0068.fits
Tags: AT_ZENITH AZEL_TARGET BIAS CAL GEMINI GMOS NON_SIDEREAL RAW SOUTH
    UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (2112, 288)    uint16
[ 1]   science                  NDAstroData       (2112, 288)    uint16
[ 2]   science                  NDAstroData       (2112, 288)    uint16
[ 3]   science                  NDAstroData       (2112, 288)    uint16
[ 4]   science                  NDAstroData       (2112, 288)    uint16
[ 5]   science                  NDAstroData       (2112, 288)    uint16
[ 6]   science                  NDAstroData       (2112, 288)    uint16
[ 7]   science                  NDAstroData       (2112, 288)    uint16
[ 8]   science                  NDAstroData       (2112, 288)    uint16
[ 9]   science                  NDAstroData       (2112, 288)    uint16
[10] 

In [21]:
# Let's reduce the data array size by a factor 8
for i in range(12):
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]

In [26]:
# write it to a test file
ad.write('../test_files/gmos_longslit_bias.fits', overwrite=True)

In [28]:
# check the header is all good
AD = astrodata.open('../test_files/gmos_longslit_bias.fits')
AD.info()

Filename: ../test_files/gmos_longslit_bias.fits
Tags: AT_ZENITH AZEL_TARGET BIAS CAL GEMINI GMOS NON_SIDEREAL RAW SOUTH
    UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (264, 36)      uint16
[ 1]   science                  NDAstroData       (264, 36)      uint16
[ 2]   science                  NDAstroData       (264, 36)      uint16
[ 3]   science                  NDAstroData       (264, 36)      uint16
[ 4]   science                  NDAstroData       (264, 36)      uint16
[ 5]   science                  NDAstroData       (264, 36)      uint16
[ 6]   science                  NDAstroData       (264, 36)      uint16
[ 7]   science                  NDAstroData       (264, 36)      uint16
[ 8]   science                  NDAstroData       (264, 36)      uint16
[ 9]   science                  NDAstroData       (264, 36)      uint16
[10]   science                  NDAstroData       (264

In [29]:
## for flat
ad = astrodata.open(DF_flat['file'].values[0])
ad.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/S20210219S0077.fits
Tags: CAL FLAT GCALFLAT GEMINI GMOS LS RAW SIDEREAL SOUTH SPECT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (2112, 288)    uint16
[ 1]   science                  NDAstroData       (2112, 288)    uint16
[ 2]   science                  NDAstroData       (2112, 288)    uint16
[ 3]   science                  NDAstroData       (2112, 288)    uint16
[ 4]   science                  NDAstroData       (2112, 288)    uint16
[ 5]   science                  NDAstroData       (2112, 288)    uint16
[ 6]   science                  NDAstroData       (2112, 288)    uint16
[ 7]   science                  NDAstroData       (2112, 288)    uint16
[ 8]   science                  NDAstroData       (2112, 288)    uint16
[ 9]   science                  NDAstroData       (2112, 288)    uint16
[10]   science   

In [30]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('../test_files/gmos_longslit_flat.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('../test_files/gmos_longslit_flat.fits')
AD.info()

Filename: ../test_files/gmos_longslit_flat.fits
Tags: CAL FLAT GCALFLAT GEMINI GMOS LS RAW SIDEREAL SOUTH SPECT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (264, 36)      uint16
[ 1]   science                  NDAstroData       (264, 36)      uint16
[ 2]   science                  NDAstroData       (264, 36)      uint16
[ 3]   science                  NDAstroData       (264, 36)      uint16
[ 4]   science                  NDAstroData       (264, 36)      uint16
[ 5]   science                  NDAstroData       (264, 36)      uint16
[ 6]   science                  NDAstroData       (264, 36)      uint16
[ 7]   science                  NDAstroData       (264, 36)      uint16
[ 8]   science                  NDAstroData       (264, 36)      uint16
[ 9]   science                  NDAstroData       (264, 36)      uint16
[10]   science                  NDAstroData       (264, 36)      u

In [31]:
## for arc
ad = astrodata.open(DF_arc['file'].values[0])
ad.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/S20210219S0101.fits
Tags: ARC CAL GEMINI GMOS LS RAW SOUTH SPECT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (2112, 288)    uint16
[ 1]   science                  NDAstroData       (2112, 288)    uint16
[ 2]   science                  NDAstroData       (2112, 288)    uint16
[ 3]   science                  NDAstroData       (2112, 288)    uint16
[ 4]   science                  NDAstroData       (2112, 288)    uint16
[ 5]   science                  NDAstroData       (2112, 288)    uint16
[ 6]   science                  NDAstroData       (2112, 288)    uint16
[ 7]   science                  NDAstroData       (2112, 288)    uint16
[ 8]   science                  NDAstroData       (2112, 288)    uint16
[ 9]   science                  NDAstroData       (2112, 288)    uint16
[10]   science                  NDAs

In [32]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('../test_files/gmos_longslit_arc.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('../test_files/gmos_longslit_arc.fits')
AD.info()

Filename: ../test_files/gmos_longslit_arc.fits
Tags: ARC CAL GEMINI GMOS LS RAW SOUTH SPECT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (264, 36)      uint16
[ 1]   science                  NDAstroData       (264, 36)      uint16
[ 2]   science                  NDAstroData       (264, 36)      uint16
[ 3]   science                  NDAstroData       (264, 36)      uint16
[ 4]   science                  NDAstroData       (264, 36)      uint16
[ 5]   science                  NDAstroData       (264, 36)      uint16
[ 6]   science                  NDAstroData       (264, 36)      uint16
[ 7]   science                  NDAstroData       (264, 36)      uint16
[ 8]   science                  NDAstroData       (264, 36)      uint16
[ 9]   science                  NDAstroData       (264, 36)      uint16
[10]   science                  NDAstroData       (264, 36)      uint16
[11]   science

In [36]:
## for bpm
ad = astrodata.open(DF_bpm['file'].values[0])
ad.info()

Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/bpm_20140601_gmos-s_Ham_22_full_12amp.fits
Tags: BPM CAL GEMINI GMOS PREPARED PROCESSED RAW SOUTH

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (2112, 256)    uint16
[ 1]   science                  NDAstroData       (2112, 256)    uint16
[ 2]   science                  NDAstroData       (2112, 256)    uint16
[ 3]   science                  NDAstroData       (2112, 256)    uint16
[ 4]   science                  NDAstroData       (2112, 256)    uint16
[ 5]   science                  NDAstroData       (2112, 256)    uint16
[ 6]   science                  NDAstroData       (2112, 256)    uint16
[ 7]   science                  NDAstroData       (2112, 256)    uint16
[ 8]   science                  NDAstroData       (2112, 256)    uint16
[ 9]   science                  NDAstroData       (2112, 256)    uint16
[10]   science

In [38]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('../test_files/gmos_longslit_bpm.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('../test_files/gmos_longslit_bpm.fits')
AD.info()

Filename: ../test_files/gmos_longslit_bpm.fits
Tags: BPM CAL GEMINI GMOS PREPARED PROCESSED RAW SOUTH

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (264, 32)      uint16
[ 1]   science                  NDAstroData       (264, 32)      uint16
[ 2]   science                  NDAstroData       (264, 32)      uint16
[ 3]   science                  NDAstroData       (264, 32)      uint16
[ 4]   science                  NDAstroData       (264, 32)      uint16
[ 5]   science                  NDAstroData       (264, 32)      uint16
[ 6]   science                  NDAstroData       (264, 32)      uint16
[ 7]   science                  NDAstroData       (264, 32)      uint16
[ 8]   science                  NDAstroData       (264, 32)      uint16
[ 9]   science                  NDAstroData       (264, 32)      uint16
[10]   science                  NDAstroData       (264, 32)      uint16
[11]   science 

In [42]:
## for object
ad = astrodata.open(DF_object['file'].values[0])
print (f"Object is {ad.object()}")
ad.info()


Object is Twilight
Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/S20210218S0029.fits
Tags: CAL GEMINI GMOS LS RAW SIDEREAL SLITILLUM SOUTH SPECT TWILIGHT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (2112, 288)    uint16
[ 1]   science                  NDAstroData       (2112, 288)    uint16
[ 2]   science                  NDAstroData       (2112, 288)    uint16
[ 3]   science                  NDAstroData       (2112, 288)    uint16
[ 4]   science                  NDAstroData       (2112, 288)    uint16
[ 5]   science                  NDAstroData       (2112, 288)    uint16
[ 6]   science                  NDAstroData       (2112, 288)    uint16
[ 7]   science                  NDAstroData       (2112, 288)    uint16
[ 8]   science                  NDAstroData       (2112, 288)    uint16
[ 9]   science                  NDAstroData       (2112, 288)    

In [43]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('../test_files/gmos_longslit_object_twilight.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('../test_files/gmos_longslit_object_twilight.fits')
AD.info()

Filename: ../test_files/gmos_longslit_object_twilight.fits
Tags: CAL GEMINI GMOS LS RAW SIDEREAL SLITILLUM SOUTH SPECT TWILIGHT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (264, 36)      uint16
[ 1]   science                  NDAstroData       (264, 36)      uint16
[ 2]   science                  NDAstroData       (264, 36)      uint16
[ 3]   science                  NDAstroData       (264, 36)      uint16
[ 4]   science                  NDAstroData       (264, 36)      uint16
[ 5]   science                  NDAstroData       (264, 36)      uint16
[ 6]   science                  NDAstroData       (264, 36)      uint16
[ 7]   science                  NDAstroData       (264, 36)      uint16
[ 8]   science                  NDAstroData       (264, 36)      uint16
[ 9]   science                  NDAstroData       (264, 36)      uint16
[10]   science                  NDAstroData       

In [44]:
## for object
ad = astrodata.open(DF_object['file'].values[1])
print (f"Object is {ad.object()}")
ad.info()


Object is AT2020caa
Filename: /Users/monika.soraisam/Desktop/tomdev/real_goats/GS-2021A-DD-102-9/S20210219S0075.fits
Tags: GEMINI GMOS LS RAW SIDEREAL SOUTH SPECT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (2112, 288)    uint16
[ 1]   science                  NDAstroData       (2112, 288)    uint16
[ 2]   science                  NDAstroData       (2112, 288)    uint16
[ 3]   science                  NDAstroData       (2112, 288)    uint16
[ 4]   science                  NDAstroData       (2112, 288)    uint16
[ 5]   science                  NDAstroData       (2112, 288)    uint16
[ 6]   science                  NDAstroData       (2112, 288)    uint16
[ 7]   science                  NDAstroData       (2112, 288)    uint16
[ 8]   science                  NDAstroData       (2112, 288)    uint16
[ 9]   science                  NDAstroData       (2112, 288)    uint16
[10]   science 

In [45]:
for i in ad.indices:
    temp = ad[i].data
    ad[i].data = temp[0:int(temp.shape[0]/8), 0:int(temp.shape[1]/8)]
# write it to a test file
ad.write('../test_files/gmos_longslit_object_at2020caa.fits', overwrite=True)
# check the header is all good
AD = astrodata.open('../test_files/gmos_longslit_object_at2020caa.fits')
AD.info()

Filename: ../test_files/gmos_longslit_object_at2020caa.fits
Tags: GEMINI GMOS LS RAW SIDEREAL SOUTH SPECT UNPREPARED

Pixels Extensions
Index  Content                  Type              Dimensions     Format
[ 0]   science                  NDAstroData       (264, 36)      uint16
[ 1]   science                  NDAstroData       (264, 36)      uint16
[ 2]   science                  NDAstroData       (264, 36)      uint16
[ 3]   science                  NDAstroData       (264, 36)      uint16
[ 4]   science                  NDAstroData       (264, 36)      uint16
[ 5]   science                  NDAstroData       (264, 36)      uint16
[ 6]   science                  NDAstroData       (264, 36)      uint16
[ 7]   science                  NDAstroData       (264, 36)      uint16
[ 8]   science                  NDAstroData       (264, 36)      uint16
[ 9]   science                  NDAstroData       (264, 36)      uint16
[10]   science                  NDAstroData       (264, 36)      uint16
